Jakki Seshapanpu(PGID-11915005) 
Manoj Sharma(PGID-11915050) 
Utkarsh Agrawal(PGID-11915020)

Task:
1. Go to Zomato.com
2. Use selenium to type in a search term, “Gachibowli”.
3. After the search result being displayed, scrape the name, cuisines, cost, timings and collections of each restaurant on the page.
4. Now navigate to each of the restaurant on the searched page and scrape the information.
5. For each restaurant collect data of reviewer, review text, reviewer metadata (which includes number of reviews, number of followers and number of photos) and review time.
6. Build a data frame containing the results as output("output_scraping.csv")

In [1]:
# Importing the libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from pandas import DataFrame

import time
import os
import csv
import numpy as np
import pandas as pd
import re

In [10]:
#Invoking the Chrome-driver
driver = webdriver.Chrome('F:/DC/data/chromedriver.exe')

In [11]:
#Opening Zomato.com via selenium
driver.get("http://www.zomato.com/hyderabad") 
time.sleep(5)
assert "Zomato" in driver.title 

In [12]:
# #Entering 'gachibowli' in search field to get list of restaurants in and around Gachibowli area
elem = driver.find_element_by_css_selector("#keywords_input")  
elem.clear()   
elem.send_keys("gachibowli")    
elem.send_keys(Keys.RETURN)    
assert "No results found." not in driver.page_source

In [13]:
# #Collecting URL's for the restaurants from the search-pages (uptill page 7) == 105 Restaurants
page = driver.current_url
Zomato_pages =[]
for j in range(1,8):
    driver.get(page + "&page=" + str(j))
    time.sleep(10)
    soup_obj = BeautifulSoup(driver.page_source,"lxml")
    title = soup_obj.find_all('a',class_='fontsize0')
    Zomato_pages = Zomato_pages + [links1.get('href') for links1 in title]
print("Following are pages we will scrap...and add to collection")
print(Zomato_pages)

Following are pages we will scrap...and add to collection
['https://www.zomato.com/hyderabad/beyond-flavours-gachibowli', 'https://www.zomato.com/hyderabad/paradise-gachibowli', 'https://www.zomato.com/hyderabad/eat-fit-gachibowli', 'https://www.zomato.com/hyderabad/shah-ghouse-hotel-restaurant-gachibowli', 'https://www.zomato.com/hyderabad/the-fishermans-wharf-gachibowli', 'https://www.zomato.com/hyderabad/over-the-moon-brew-company-gachibowli', 'https://www.zomato.com/hyderabad/shah-ghouse-spl-shawarma-gachibowli', 'https://www.zomato.com/hyderabad/hyper-local-gachibowli', 'https://www.zomato.com/hyderabad/abs-absolute-barbecues-gachibowli', 'https://www.zomato.com/hyderabad/flechazo-gachibowli', 'https://www.zomato.com/hyderabad/absolute-sizzlers-gachibowli', 'https://www.zomato.com/hyderabad/cream-stone-gachibowli', 'https://www.zomato.com/hyderabad/sardarjis-chaats-more-gachibowli', 'https://www.zomato.com/hyderabad/the-lal-street-bar-exchange-gachibowli', 'https://www.zomato.com/

In [14]:
# Creates a new .csv file that the scraped data will be written to
csv_file = open('Zomato_reviews_test.csv', 'w', encoding="UTF-8", newline="")

# Defining Writer and Column values for the CSV file
writer = csv.writer(csv_file)
writer.writerow(
    ['Rest_Name', 'Rest_Cuisines','Rest_Cost_for_two','Rest_Cost_for_beer','Rest_Timings','Overall_ratings', 'Review_writer',
      'review_date', 'reviewer_No_Photos','reviewer_metadata', 'customer_review_rating', 'customer_review_text'])

198

In [ ]:
#Outer for loop iterates through all the restaurants pages

for page in Zomato_pages:
    driver.get(page)
    time.sleep(10)
    try:
        print("=" * 40)  # Display the Restaurant page which is being scraped
        print("Scraping " + page) 
        # Click on All reviews link on the home-page for restaurant        
        try:
            elm = driver.find_element_by_xpath('//a[@data-sort="reviews-dd"][contains(.,"All Reviews")]')
            elm.click()
            time.sleep(10)
            print("successful click on All Reviews")
            for i in range(0,19):
                load_reviews = driver.find_element_by_xpath(
                    '//span[@class="zred"][contains(.,"Load More")]')
                load_reviews.location_once_scrolled_into_view
                try:
                    load_reviews.click()
                    time.sleep(10)
                    print("successful click on Load Reviews")
                except Exception as e:
                    print(e)
                    print("Unsuccessful click on Load Reviews")
        except Exception as e:
            print(e)
            print("Unsuccessful click on All Reviews")
        reviews = driver.find_elements_by_xpath(
                         '//*[@id="reviews-container"]//div [@data-snippet="restaurant-review"]')
        print("Total number of reviews collected..", len(reviews))
        for review in reviews:
            review_dict = {}
            try:
                        Rest_Name = review.find_element_by_xpath('//div [@class="col-l-12"]/h1').text
            except:
                        Rest_Name = page
            try:
                        Rest_Cuisines = review.find_element_by_xpath('//div[@class="res-info-cuisines clearfix"]').text
            except:
                        Rest_Cuisines = ""
            try:
                        Rest_Cost_for_two = review.find_element_by_xpath(
                            '//div[@class="res-info-detail"]//span[@tabindex="0"]').text
                        Rest_Cost_for_two = re.sub('\n|[a-z]|[).(â‚¹]','',Rest_Cost_for_two)
            except:
                        Rest_Cost_for_two = ""
            try:
                        Rest_Cost_for_beer = review.find_element_by_xpath(
                            '//div[@class="mt5"]//span[@class=""]').text
            except:
                        Rest_Cost_for_beer = "No Beer"
            try:
                        Rest_Timings = review.find_element_by_xpath(
                            '//div[@class="res-info-timings"]//div[@class="medium"]').text
                        Rest_Timings = re.sub('\n|[â€“]','-',Rest_Timings)
            except:
                        Rest_Timings = ""
            try:
                        Overall_ratings = review.find_element_by_xpath('//div [@class="col-l-4 tac left"]//div[@tabindex="0"]').text
            except:
                        Overall_ratings = ""
            try:
                        Review_writer = review.find_element_by_xpath(
                            './/div[@class="header nowrap ui left"]').text
            except:
                        Review_writer = ""
            try:
                        review_date = review.find_element_by_xpath(
                            './/a[@class="grey-text"]').text
            except:
                        review_date = ""
            try:
                        reviewer_metadata = review.find_element_by_xpath(
                            './/span[@class="grey-text fontsize5 nowrap"]').text
            except:
                        reviewer_metadata = ""
            
            try:
                        reviewer_No_Photos = review.find_elements_by_xpath(
                            './/img[@class="margin0 ui card image"]')
                        reviewer_No_Photos = len(reviewer_No_Photos)
            except:
                        reviewer_No_Photos = 0
            try:
                        customer_review_rating = review.find_element_by_xpath(
                        './/div[contains(@class,"rev-text ")]//div[contains(@class,"ttupper")]').get_attribute('aria-label')

                        customer_review_text = review.find_element_by_xpath(
                        './/div[contains(@class,"rev-text ")]').text
                        customer_review_text = re.sub('\n|RATED','', customer_review_text)
            except:
                        customer_review_text = ""
                        customer_review_rating = ""

           # Write contents for dictionary to csv file
            review_dict['Rest_Name'] = Rest_Name
            review_dict['Rest_Cuisines'] = Rest_Cuisines
            review_dict['Rest_Cost_for_two'] = Rest_Cost_for_two
            review_dict['Rest_Cost_for_beer'] = Rest_Cost_for_beer
            review_dict['Rest_Timings'] = Rest_Timings
            review_dict['Overall_ratings'] = Overall_ratings
            review_dict['Review_writer'] = Review_writer
            review_dict['review_date'] = review_date
            review_dict['reviewer_No_Photos'] = reviewer_No_Photos
            review_dict['reviewer_metadata'] = reviewer_metadata
            review_dict['customer_review_rating'] = customer_review_rating
            review_dict['customer_review_text'] = customer_review_text       

            writer.writerow(review_dict.values())
    except Exception as e:
        print(e)
        csv_file.close()
        driver.close()
        break
csv_file.close()
driver.close()

Scraping https://www.zomato.com/hyderabad/beyond-flavours-gachibowli
successful click on All Reviews
successful click on Load Reviews
successful click on Load Reviews
successful click on Load Reviews
successful click on Load Reviews
successful click on Load Reviews
successful click on Load Reviews
successful click on Load Reviews
successful click on Load Reviews
successful click on Load Reviews
successful click on Load Reviews
successful click on Load Reviews
successful click on Load Reviews
successful click on Load Reviews
successful click on Load Reviews
successful click on Load Reviews
successful click on Load Reviews
successful click on Load Reviews
successful click on Load Reviews
successful click on Load Reviews
Total number of reviews collected.. 105
Scraping https://www.zomato.com/hyderabad/paradise-gachibowli
successful click on All Reviews
successful click on Load Reviews
successful click on Load Reviews
successful click on Load Reviews
successful click on Load Reviews
succes